In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import missingno as msno

os.environ["MODIN_ENGINE"] = "ray"  # Modin will use Ray

# import pandas as pd
import modin.pandas as pd
import ray
ray.init(ignore_reinit_error=True)

import warnings
warnings.filterwarnings("ignore")

In [2]:
%run ./module_P2_utils.ipynb

> important notice : import is made with date type parsing specified for date columns, discovered after preliminary analysis

<center><img src="assets/loading.png" alt="drawing" width="500"/></center>

Loading the dataset from csv file


In [ ]:
df = pd.read_csv('en.openfoodfacts.org.products.csv', sep='\t', parse_dates=['created_datetime','last_modified_datetime'])

In [ ]:
# df_bck = df.copy()

In [ ]:
# df = df_bck.copy()

In [ ]:
print(f"dataframe have {df.shape[1]} columns and {df.shape[0]} rows")

In [ ]:
df.head()

## Content Discovery

What's inside the dataset ?

In [ ]:
df.info()

In [ ]:
for col in range(0, df.columns.size):
    print(f'{df.columns[col]:<45} {str(df.dtypes[col]):>}')

In [ ]:
plt.figure(figsize=(10,10))
df.dtypes.value_counts().plot.pie(title="repartition of features type in dataset")

Most of the features are qualitative or quantitative. 
Just a few of them are date / time based or integers

<center><img src="assets/data_cleaning.jpeg" alt="drawing" width="500"/></center>

# CLEANING

In this Chapter, we will:
* investigate minority categories
* remove empty features
* remove some of the features (columns) that don't have much interest.
* remove obvious and non sense values


## Minority Categories : Investigate `int` and `date` features


In [ ]:
df.columns[((df.dtypes != 'float') & (df.dtypes != 'object'))]

In [ ]:
df.dtypes[['created_t','created_datetime','last_modified_t', 'last_modified_datetime']]

In [ ]:
df[['created_t', 'created_datetime', 'last_modified_t', 'last_modified_datetime']].head()

In [ ]:
pd.to_datetime(df['created_t'], unit='s').head()

As we can see, columns are the same between X_datetime and X_t, so we choose to remove one of them

In [ ]:
df.drop({
    'created_t', 
    'last_modified_t', 
        }, axis=1, inplace=True)

## Removal of Empty features

let's compute the ratio: $ \frac{\text{empty values}}{\text{nb rows}} $

In [ ]:
df.sample(round(df.shape[0]*0.02)).isna().mean().sort_values().plot(kind="hist")

About 120 out of 186 columns have more than 80% of empty cells. The dataset despite being large, is almost empty !

In [ ]:
msno.matrix(df.sample(round(df.shape[0]*0.02)))

Taking a sample (20%) of the dataset, to ease compute, we can see there are some plateau of columns missing data, let's consider we want to remove at least those that have more than 70% of missing value.

In [ ]:
threshold = 0.7
df = df[df.columns[df.isna().mean() < threshold]]

## Removal of Not Interesting features

Some features provides values that won't be of interest for future analysis, we remove them manually.

For instance, we decide that features providing `url` are removed

In [ ]:
url_columns = df.filter(regex='url').columns.values
print(url_columns)
df = df.drop(url_columns, axis=1)

## Obvious Out of Range Values

Provided file [input/data-fields.txt]() explains some of the features maximum and minimum values. For instance most of values `*_100g` have boundaries [0; 100].

> - fields that end with _100g correspond to the amount of a nutriment (in g, or kJ for energy) for 100 g or 100 ml of product

So, values out of this boundaries should be removed, for instance we can't have more than 100g of fat in a portion of 100g.

Despite:
* `energy*` values are not to be considered the same since the unit is not the same (kJ per 100g)
* `nutrition_score_fr_100g` hasn't to be considered the same since it is a score, not a ratio of ingredient in a portion

Documentation also states that quantitative values `*_n` are not continuous, therefore we should consider them as discrete and categorical features.

Finally, for continuous values, we may replace NaN values by 0.0

In [ ]:
cols100g = df.filter(regex="_100g").columns

cols100g_bounded = cols100g.drop(['energy-kcal_100g', 'energy_100g', 'nutrition-score-fr_100g'])
cols100g_bounded

### Fill Nan Values

For **continuous** quantitative features

In [ ]:
df.select_dtypes('float').columns

In [ ]:
quantis_nullable =['nutriscore_score', 'nova_group']
quantis_non_nullable = df.select_dtypes('float').columns.drop(quantis_nullable)

In [ ]:
df.fillna({col:0.0 for col in quantis_non_nullable}, inplace=True)

### Remove Out of Range Values

In [ ]:
rogues = pd.DataFrame(
    [
        (df[cols100g_bounded] < 0).sum(),
        (df[cols100g_bounded] > 100).sum(),
        ((df[cols100g_bounded] < 0).sum() + (df[cols100g_bounded] > 100).sum())
    ],
    index=['negatives', 'greater_100', 'to_remove'],
    columns=cols100g_bounded
)
rogues.loc['Column_Total']= rogues.sum(numeric_only=True, axis=0)
rogues.loc[:,'Row_Total'] = rogues.sum(numeric_only=True, axis=1)
rogues

As we can see, just a few values are inconsistent and one can consider removing them wouldn't be necessary. This is misleading, since those values may have a great impact on the features' distribution.

Now the question is : *"What to do with these values ?"*

Several options exists:
* removing the row which contains such value
* replacing the value with NaN ? or with another value ?

This decision must be done carefully because either we may end up to remove lots of rows, or we would be modifying the 'meaning' of the data.

Let's investigate the two options

#### 1. Removing rows


In [ ]:
# tmpdf = df.copy()
# df = tmpdf.copy()

In [ ]:
initial_size = df.shape[0]
print(initial_size)
for col in cols100g_bounded:
    df = df[(df[col] <= 100) & (df[col] >= 0)]
final_size = df.shape[0]
print(final_size)
print(f"removed {initial_size - final_size} rows")

#### 2. Replacing with arbitrary values

Since removal above didn't had too much impacts on the dataset, we don't have to prospect this option.

## Remove duplicates

We use `code` as the primary key in our dataset, thus we expect no duplicate values on this column

In [ ]:
df.loc[df.duplicated(['code'])]

About 500 values will be removed, very few compared to the size of the dataset

In [ ]:
df.drop_duplicates('code', inplace=True)

## Results of cleaning

In [ ]:
df.shape

In [ ]:
df.dtypes.value_counts().plot.pie()

In [ ]:
df.isna().mean().sort_values().plot(kind="bar", title='percentage of missing values per feature', figsize=(10,10))

In [ ]:
pd.DataFrame({'type': df.dtypes, 'percent_missing_value': df.isna().mean() * 100}).sort_values(by=['percent_missing_value'])

# Conclusion on Exploratory Data Analysis

## What's provided ?

This dataset has a mix of different kinds of data.

### Size / Nb lines & columns

It has almost 2 Mio raw values with 186 features. despite, after cleaning only 37 columns remains.

### Features Comprehension

It contains a mix of types : dates, strings, numbers

       
#### Qualitative / Categorical
code
creator
created_datetime
last_modified_datetime
product_name
brands
brands_tags
categories
categories_tags
categories_en
countries
countries_tags
countries_en
ingredients_text

##### Ordinal Categories
additives_n
ingredients_from_palm_oil_n
ingredients_that_may_be_from_palm_oil_n

nutriscore_score
nutriscore_grade

nova_group [explanation (fr)](https://www.thierrysouccar.com/nutrition/info/nova-une-classification-des-aliments-basee-sur-la-science-3936)
pnns_groups_1 [explanation (fr)](https://solidarites-sante.gouv.fr/IMG/pdf/pnns4_2019-2023.pdf)
pnns_groups_2

states
states_tags
states_en
main_category
main_category_en

#### Quantitative

energy-kcal_100g
energy_100g
fat_100g
saturated-fat_100g
carbohydrates_100g
sugars_100g
proteins_100g
salt_100g
sodium_100g
nutrition-score-fr_100g


In [ ]:
# df.to_csv("./tmp/df_01_cleaned.csv")
df.to_pickle("./tmp/df_01_cleaned.gzip")